In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pytest
from funcs.geometry import (create_spherical_grid,
                            set_up_oblique_auroral_ring,
                            rotate_around_arb_axis)


from funcs.analytical import (v_phi,
                              get_analytical_spectral_line,)

# dark background
plt.style.use('dark_background')



from funcs.auroralring import AuroralRing





In [ ]:

# set up the ring
ring = AuroralRing(i_rot=30/180*np.pi, i_mag=10 * np.pi/180, latitude=41.5*np.pi/180,
                   width=1 * np.pi/180, Rstar=1, P_rot=1.5 * np.pi, N=100, 
                   norm=11, gridsize=int(4e5), v_bins=None)

# calculate the flux
alpha = 10 * np.pi/180

flux_analytical = ring.get_flux_analytically(alpha)

# calculate the flux 
# this serves as measurement
flux_numerical = ring.get_flux_numerically(alpha)
flux_err = np.ones_like(flux_numerical) * 0.05

plt.plot(ring.v_mids, flux_analytical, label='analytical')
plt.plot(ring.v_mids, flux_numerical, label='numerical')
plt.legend()


In [ ]:
fig = plt.figure(figsize=(10, 5))
spec = fig.add_gridspec(ncols=1, nrows=1)

ax = fig.add_subplot(spec[0, 0], projection='3d')
ax.set_axis_off()

ring.plot_sphere_with_auroral_ring(ax, alpha)

ring.plot_layout_sphere(ax, view="observer front")

In [ ]:

# set up the ring
ring = AuroralRing(i_rot=30/180*np.pi, i_mag=10 * np.pi/180, latitude=40*np.pi/180,
                   width=1 * np.pi/180, Rstar=1, P_rot=1.5 * np.pi, N=100, 
                   norm=11, gridsize=int(4e5))

# calculate the flux
alpha = 15 * np.pi/180

flux_analytical = ring.get_flux_analytically(alpha)

# calculate the flux 
# this serves as measurement
flux_numerical = ring.get_flux_numerically(alpha)
flux_err = np.ones_like(flux_numerical) * 0.02

plt.plot(ring.v_mids, flux_analytical, label='analytical')
plt.plot(ring.v_mids, flux_numerical, label='numerical')
plt.legend()




In [ ]:
fig = plt.figure(figsize=(10, 5))
spec = fig.add_gridspec(ncols=1, nrows=1)

ax = fig.add_subplot(spec[0, 0], projection='3d')
ax.set_axis_off()

ring.plot_sphere_with_auroral_ring(ax, alpha)

ring.plot_layout_sphere(ax, view="observer front")

In [ ]:
def log_likelihood(theta, bins, flux_numerical, flux_err, phi, omega, Rstar, i_rot, i_mag):
    latitude, alpha = theta
    model = get_analytical_spectral_line(phi, i_rot, i_mag, latitude, alpha,bins, omega, Rstar, norm=10)
    sigma2 = flux_err**2 + model**2 
    return -0.5 * np.sum((flux_numerical - model) ** 2 / sigma2 + np.log(sigma2))

In [ ]:
from scipy.optimize import minimize

nll = lambda *args: -log_likelihood(*args)
initial = np.array([ring.latitude, alpha]) + 0.1 * np.random.randn(2)
soln = minimize(nll, initial, args=(ring.v_bins, flux_numerical, flux_err, ring.phi, 
                                    ring.omega, ring.Rstar, ring.i_rot, ring.i_mag))
l, a = soln.x

print("Maximum likelihood estimates:")
# print("r = {0:.3f}".format(r*180/np.pi))
# print("m = {0:.3f}".format(m*180/np.pi))
print("l = {0:.3f}".format(l*180/np.pi))
print("a = {0:.3f}".format(a*180/np.pi))

In [ ]:
def log_prior(theta):
    latitude, alpha = theta
    if  0 < latitude < np.pi/2 and 0 < alpha < np.pi*2:
        return 0.0
    return -np.inf


def log_probability(theta, x, y, yerr, phi, omega, Rstar, i_rot, i_mag):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr, phi, omega, Rstar, i_rot, i_mag)

In [ ]:
import emcee

pos = soln.x + 1e-2 * np.random.randn(32, 2)
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_probability, args=(ring.v_bins, flux_numerical, flux_err, ring.phi,
                                           ring.omega, ring.Rstar, ring.i_rot, ring.i_mag))
sampler.run_mcmc(pos, 5000, progress=True);

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 5), sharex=True)
samples = sampler.get_chain()
labels = ["lat", "alpha"]

for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i]*180/np.pi, "w", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=400, thin=15, flat=True)

import corner

fig = corner.corner(
    flat_samples*180/np.pi, labels=labels, truths=np.array([ring.latitude, alpha])*180/np.pi,
    );

In [ ]:
# --- SET UP the numerical solution

THETA, PHI = create_spherical_grid(int(1e5))
# input parameters

# rotation period in days
P_rot = 0.5 * np.pi

# stellar radius in solar radii
Rstar = 1.

# upper latitude of ring around magnetic axis in radians
PHI_max = 42 * np.pi/180

# lower latitude of ring around magnetic axis in radians
PHI_min = 41 * np.pi/180

# lower latitude must be smaller than upper latitude
assert PHI_min < PHI_max

# convert dxr_visible from km/s to nm using speed of light
# and wavelength of the line
c = 299792.458 # km/s
wavelength = 3825.4 # AA Ca II H line


#  SET UP the analytical solution

phi = np.linspace(0, 2*np.pi, 1500)

# inclination of rotation axis in radians with the right convention
i_rot = np.pi/2 - 80 * np.pi/180

# inclination of magnetic axis in radians relative to rotation axis
i_mag = 10 * np.pi/180

# latitude of the ring in radians
latitude = PHI_min#(PHI_max + PHI_min) / 2

# angular velocity in rad/d
omega = 2 * np.pi / P_rot

# phase
alpha = 110 * np.pi/180

# velocity bins
bins = np.linspace(-50,50,100)
binmids = (bins[1:] + bins[:-1])/2


# NUMERICAL solution ------

(x, y, z), z_rot, z_rot_mag = set_up_oblique_auroral_ring(THETA, PHI, PHI_max, PHI_min, i_rot, i_mag)

xr, yr, zr = rotate_around_arb_axis(alpha, np.array([x, y, z]), z_rot)

numerical_line = numerical_spectral_line(alpha, x, y, z, z_rot, omega, Rstar, bins, dalpha=.1 * np.pi/180)

# -------------------------




# FIGURE SETUP ------

# set up a figure with six panels in a 3x2 grid, the first panel is in 3d projection
fig = plt.figure(figsize=(10, 5))
spec = fig.add_gridspec(ncols=2, nrows=1, width_ratios=[1, 2])

ax = fig.add_subplot(spec[0, 0], projection='3d')
ax.set_axis_off()
ax1 = fig.add_subplot(spec[0, 1])

# --------------------



# THE SPHERE ---------

ax.scatter(np.sin(THETA)*np.cos(PHI),
              np.sin(THETA)*np.sin(PHI),
              np.cos(THETA), c='grey', alpha=0.01)

# plot the x axis as a dashed line
ax.plot([-1, 1], [0, 0], [0, 0], c='w', ls='--')

z_mag_alpha = rotate_around_arb_axis(alpha, z_rot_mag, z_rot)

# plot z_rot
ax.plot([0, 1.5 *z_rot[0]], [0, 1.5 *z_rot[1]], [0,1.5 * z_rot[2]], c='r')


# plot z_rot_mag
ax.plot([0, z_mag_alpha[0]], [0, z_mag_alpha[1]], [0, z_mag_alpha[2]], c='yellow')

# THE RING ----------

# plot the rotated blue points
ax.scatter(xr, yr, zr, alpha=1, s=1)

# --------------------



# LAYOUT -------------

# set figure limits to -1.5 to 1.5 on eacha xis
ax.set_xlim(-1.2, 1.2)
ax.set_ylim(-1.2, 1.2)
ax.set_zlim(-1., 1.)

# label axes
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# rotate the figure such that x-axis point towards me
ax.view_init(0, 90)

# legend
ax.set_title(f'rotational phase {alpha*180/np.pi:.0f} deg')

# let axes disappear
ax.set_axis_off()

# --------------------




# ANALYTICAL solution

for linestyle, label, latitude in zip(
                    ['-.', '-', ':'],
                    ["min lat.", "max lat.", "mid lat."], 
                    [PHI_min, PHI_max, (PHI_max + PHI_min)/2]):
    
    analytical_line = get_analytical_spectral_line(phi, i_rot, i_mag, latitude, alpha, bins, omega, Rstar)
            


    ax1.plot(binmids, analytical_line, label=f"analytical {label} {latitude/np.pi*180:.0f} deg", linestyle=linestyle)



ax1.plot(binmids, numerical_line, zorder=-10, linestyle="-", label="numerical", linewidth=3)
ax1.set_xlabel(r'velocity $v$ [km/s]')
ax1.set_ylabel(r'$F(v_x)$')
ax1.set_xlim(-50, 50)
ax1.set_ylim(0,1.4)
ax1.legend(frameon=False, loc='upper left')





In [ ]:
phi = np.linspace(0, 2*np.pi, 150)

# inclination of rotation axis in radians with the right convention
i_rot = np.pi/2 - 80 * np.pi/180

# inclination of magnetic axis in radians relative to rotation axis
i_mag = 70 * np.pi/180

# latitude of the ring in radians
latitude = 60 * np.pi/180

# alpha=np.pi/3

# go through phi_bound

# fig, axes = plt.subplots(1,1,figsize=(10, 10))
# axes = axes.flatten()

fluxes = []
vs = []

for alpha in np.linspace(0, 2*np.pi,100):
    res = get_spectral_line(phi, i_rot, i_mag, latitude, alpha)
    print(alpha)
    phi_bounds, phi_bounds_v, signum, v_bounds_phi, v_bounds, flux, fluxmask, mask, v, phi, A, B, C = res

    fluxes.append((flux*fluxmask)[mask])
    vs.append(v[mask])

# make a dataframe from vs and fluxes
df = pd.DataFrame({'v': np.concatenate(vs), 'flux': np.concatenate(fluxes)})

# bin in v space and sum fluxes
df_binned = df.groupby(pd.cut(df.v, bins=100)).sum()

# plot
df_binned.index = pd.IntervalIndex(df_binned.index).mid

plt.plot(df_binned.index.values,  df_binned.flux.values)
plt.ylim(0,)

In [ ]:
# set up a figure with six panels in a 3x2 grid, the first panel is in 3d projection
fig, axes = plt.subplots(3,2,figsize=(15, 10))
axes = axes.flatten()

# add the 3d projection to the top panel
ax = fig.add_subplot(321, projection='3d')

axes[0].set_axis_off()

ax2 = ax4

ax3 = ax1

## Archive

In [ ]:

v = np.linspace(-5, 5, 200)

phi1, phi2 = phi_v(v, X, Y, Z)

plt.plot(v, phi1, linewidth=6, label=r'$\phi_1(v)$')
plt.plot(v, phi2, linewidth=4, label=r'$\phi_2(v)$')


phi = np.linspace(0, 2*np.pi, 100)

v1 = v_phi(phi, X, Y, Z)

plt.plot(v1, phi, linestyle='--', c="r", label=r'$v(\phi)$')

#fill between phi 5 and 5.5 horizontal
plt.fill_between(v, 5, 6, color='orange', alpha=0.4)
plt.fill_between(v, 0, 1.7, color='orange', alpha=0.4, label="integration bounds")


plt.xlabel(r'$v$')
plt.ylabel(r'$\phi$')
plt.legend(frameon=False, loc=(.5,.4), fontsize=14)
# plt.xlim(-3, 5)
plt.ylim(0, 2*np.pi);

In [ ]:
def get_v_bounds(X, Y, Z):

    phi_where_v_turns = [np.arctan(Y/Z)]

    if phi_where_v_turns[0] < np.pi:
        phi_where_v_turns.append(np.arctan(Y/Z) + np.pi)
    elif phi_where_v_turns[0] > np.pi:
        phi_where_v_turns.append(np.arctan(Y/Z) - np.pi)

    for i in range(2):
        if phi_where_v_turns[i] < 0:
            print("phi < 0")
            phi_where_v_turns[i] = phi_where_v_turns[i] + 2 * np.pi
        elif phi_where_v_turns[i] > 2 * np.pi:
            print("phi > 2pi")
            phi_where_v_turns[i] = phi_where_v_turns[i] - 2 * np.pi

    zero_v = v_phi(phi_where_v_turns, X, Y, Z)    

    signum = np.sign( - np.sin(phi_where_v_turns) * Y - np.cos(phi_where_v_turns) * Z)
    
    return phi_where_v_turns, zero_v, signum

def get_phi_bounds(phi, A, B, C, X, Y, Z):

    a, b = B/A, C/A

    bounds = []
    for n in range(-2, 3):

        _ = a**2 + b**2 - 1 
        
        if _ > 0:
            b1 = 2 * (np.arctan((a - np.sqrt(_)) / (b - 1)) + n * np.pi)
            if (b1 > 0) & (b1 < 2 * np.pi):
                bounds.append(b1)
            b2 = 2 * (np.arctan((a + np.sqrt(a**2 + b**2 - 1)) / (b - 1)) + n * np.pi)
            if (b2 > 0) & (b2 < 2 * np.pi):
                bounds.append(b2)

    # derivative of the function gets us which side of the bound to include
    bounds = np.sort(np.array(bounds))

    # get the derivative dx/dphi
    dx_dphi_at_bounds = np.sign(dx_dphi(bounds, B, C))

    # print(bounds, dx_dphi_at_bounds)

    def bounds_mask(phi):

        default = (phi < 2 * np.pi) & (phi > 0)
        mark = 1 # we start inclusive 

        for bound, dx_dphi_at_bound in zip(bounds, dx_dphi_at_bounds):

            if dx_dphi_at_bound == 1:

                if mark == 1:
                    # print("1")
                    default = default & (phi >bound)
                    mark = 1
                    # print("We are already inclusive, but the slope is positive, "
                    #       "so we must be at the beginning of the full interval")

                elif mark == 0:
                    # print("2")
                    default = default | (phi > bound)
                    mark = 1
                    # print("The slope has been negative, changes to positive, "
                    #       "so we must be at the beginning of an included interval")

            elif dx_dphi_at_bound == -1:

                if mark == 1:
                    # print("3")
                    default = default & (phi < bound)
                    mark = 0
                    # print("We are already inclusive, but the slope is negative, "
                    #       "so we must be at the end of the included interval.")

                elif mark == 0:
                    raise ValueError('negative slope although we already have a negative slope')

        return default
    
    bounds_v = v_phi(bounds, X, Y, Z)

    return bounds, bounds_v, dx_dphi_at_bounds, bounds_mask(phi)


def phi_v(v, X, Y, Z):
    n, m = 1, 0
    phi1 = 2 * (np.arctan((Y - np.sqrt(-v**2 + 2 * v * X - X**2 + Y**2 + Z**2)) / (v - X + Z) ) + n * np.pi)
    phi2 = 2 * (np.arctan((Y + np.sqrt(-v**2 + 2 * v * X - X**2 + Y**2 + Z**2)) / (v - X + Z) ) + m * np.pi)

    return phi1, phi2



def dx_dphi(phi, B, C):
    return B * np.cos(phi) - C * np.sin(phi)

In [ ]:
24*3600